# Annotating Data & Training Custom NER model for Suicide Risk Prediction

In this tutorial, we'll see how we can easily annotate data using pre-defined vocabulary / key-word matching, and upload them as pre-annotations to NLPLab.

We'll be using the AnnotationLab module to create, configure, export and import projects using the AnnotationLab module with minimal code.

Note: The Annotation Lab module is available in Spark NLP for Healthcare 4.2.1+

## 0. Initial Configurations

In [0]:
import pandas as pd
import os

import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.base import *
from sparknlp_jsl.annotator import *

from pyspark.ml import Pipeline, PipelineModel
from sparknlp.training import CoNLL

print('sparknlp.version : ',sparknlp.version())
print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp.version : 4.2.1
sparknlp_jsl.version : 4.2.1
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 spark://10.139.64.5:7077 
 AppName 
 Databricks Shell

For this exercise, we define two different layers of solutions:
1. Bronze: Text Matcher based rudimentary results - that will be uploaded in annotation lab and refined.
2. Silver: After annotating the documents properly in Annotation Lab, train an NER model, get results

In [0]:

delta_bronze_path='/FileStore/HLS/nlp/delta/bronze/'

dbutils.fs.mkdirs(delta_bronze_path)

os.environ['delta_bronze_path']=f'/dbfs{delta_bronze_path}'

delta_silver_path='/FileStore/HLS/nlp/delta/silver/'

dbutils.fs.mkdirs(delta_silver_path)

os.environ['delta_silver_path']=f'/dbfs{delta_silver_path}'


Download Vocabulary files for Suicide Detection

In [0]:
%sh

cd $delta_bronze_path

wget https://github.com/JohnSnowLabs/spark-nlp-workshop/raw/master/databricks/python/healthcare_case_studies/data/suicide_pred_vocab_data.zip

unzip suicide_pred_vocab_data.zip

Archive: suicide_pred_vocab_data.zip

In [0]:
data_vocab_path = f"{delta_bronze_path}suicide_pred_vocab_data/"

dbutils.fs.ls(data_vocab_path)

Out[4]: [FileInfo(path='dbfs:/FileStore/HLS/nlp/delta/bronze/suicide_pred_vocab_data/suicide_behavior_vocab.txt', name='suicide_behavior_vocab.txt', size=7654),
 FileInfo(path='dbfs:/FileStore/HLS/nlp/delta/bronze/suicide_pred_vocab_data/suicide_det_data.csv/', name='suicide_det_data.csv/', size=0),
 FileInfo(path='dbfs:/FileStore/HLS/nlp/delta/bronze/suicide_pred_vocab_data/suicide_others_vocab.txt', name='suicide_others_vocab.txt', size=2590),
 FileInfo(path='dbfs:/FileStore/HLS/nlp/delta/bronze/suicide_pred_vocab_data/suicide_psychache_vocab.txt', name='suicide_psychache_vocab.txt', size=5569),
 FileInfo(path='dbfs:/FileStore/HLS/nlp/delta/bronze/suicide_pred_vocab_data/suicide_trauma_vocab.txt', name='suicide_trauma_vocab.txt', size=5578)]

### Running Spark NLP Pipeline

In [0]:
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentenceDetector = SentenceDetector()\
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
  .setInputCols("sentence")\
  .setOutputCol("token")

text_matcher1 = TextMatcher().setInputCols("sentence","token").setOutputCol("textmatch_chunk1").setEntityValue("SUICIDE_BEHAVIOR")\
    .setEntities(data_vocab_path+"suicide_behavior_vocab.txt").setCaseSensitive(False).setMergeOverlapping(True)\
    .setBuildFromTokens(True)

text_matcher2 = TextMatcher().setInputCols("sentence","token").setOutputCol("textmatch_chunk2").setEntityValue("SUICIDE_PSYCHACHE")\
    .setEntities(data_vocab_path+"suicide_psychache_vocab.txt").setCaseSensitive(False).setMergeOverlapping(True)\
    .setBuildFromTokens(True)

text_matcher3 = TextMatcher().setInputCols("sentence","token").setOutputCol("textmatch_chunk3").setEntityValue("SUICIDE_TRAUMA")\
    .setEntities(data_vocab_path+"suicide_trauma_vocab.txt").setCaseSensitive(False).setMergeOverlapping(True)\
    .setBuildFromTokens(True)

text_matcher4 = TextMatcher().setInputCols("sentence","token").setOutputCol("textmatch_chunk4").setEntityValue("SUICIDE_OTHERS")\
    .setEntities(data_vocab_path+"suicide_others_vocab.txt").setCaseSensitive(False).setMergeOverlapping(True)\
    .setBuildFromTokens(True)

chunk_merger = ChunkMergeApproach()\
    .setInputCols(['textmatch_chunk1', 'textmatch_chunk2', 'textmatch_chunk3', 'textmatch_chunk4'])\
    .setOutputCol("all_chunks")

pipeline =  Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        text_matcher1, text_matcher2, text_matcher3, text_matcher4,
        chunk_merger
    ]
)

p_model = pipeline.fit(spark.createDataFrame(pd.DataFrame({'text': ['']})))

l_model = LightPipeline(p_model)

/databricks/spark/python/pyspark/sql/pandas/conversion.py:499: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
 arrow_data = [[(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]

In [0]:
data = spark.read.csv(data_vocab_path+'suicide_det_data.csv').withColumnRenamed('_c0', 'text')
print (data.count())
data.show(3)


2667
+--------------------+
 text|
+--------------------+
I really don't wa...|
the answer is non...|
to whoever read t...|
+--------------------+
only showing top 3 rows

In [0]:
results = p_model.transform(data).collect()

### Results

In [0]:
from sparknlp_display import NerVisualizer

displayHTML(NerVisualizer().display(results[0], 'all_chunks', return_html=True))

I really don't want to live SUICIDE_BEHAVIOR anymore - i've never talked about this with anyone, but i really don't want to live SUICIDE_BEHAVIOR anymore. because what is the point? why SUICIDE_TRAUMA am i here? i can't SUICIDE_TRAUMA do anything useful in my life SUICIDE_BEHAVIOR until i become an adult, i ruin almost everything, i cut SUICIDE_BEHAVIOR a lot and am always worried someone will SUICIDE_BEHAVIOR see the scars SUICIDE_TRAUMA . i get confused SUICIDE_PSYCHACHE when i see people genuinely enjoying themselves and having fun. it blows my mind SUICIDE_PSYCHACHE . how are they so happy? how can anyone love SUICIDE_TRAUMA themselves? i don't love SUICIDE_TRAUMA myself. i'm fact i hate myself SUICIDE_TRAUMA . because i deserve it. what use am i to anyone?

## Upload Pre-annotations to Annotation Lab

In [0]:
# annotation lab module
from sparknlp_jsl.alab import AnnotationLab

alab = AnnotationLab()

# Set Credentials
username=''
password=''
client_secret="", 
annotationlab_url=""

alab.set_credentials(

  # required: username
  username=username,

  # required: password
  password=password,

  # required: secret for you alab instance (every alab installation has a different secret)
  client_secret=client_secret,

  # required: http(s) url for you annotation lab
  annotationlab_url=annotationlab_url
)

Convert Predictions to Pre-annotations

In [0]:
pre_annotations, summary = alab.generate_preannotations(all_results = results, document_column = 'document', ner_columns = ['all_chunks'])

Processing 2667 Annotations.

In [0]:
## Summary helps identify how many types of entities, assertions, and relations are present in the results - this is useful for setting project config.
summary

Out[11]: {'ner_labels': ['SUICIDE_BEHAVIOR',
 'SUICIDE_TRAUMA',
 'SUICIDE_OTHERS',
 'SUICIDE_PSYCHACHE'],
 'assertion_labels': [],
 're_labels': []}

Create Project, Set Configuration & Upload

In [0]:
# create new project
alab.create_project('suicide_detection')

Operation completed successfully. Response code: 201
Out[21]: {'project_name': 'suicide_detection'}

In [0]:
# set configuration
alab.set_project_config(
  project_name = 'suicide_detection',
  ner_labels = summary['ner_labels'])

Operation completed successfully. Response code: 201
Out[22]: {'messages': [{'message': 'Project config saved.', 'success': True}]}

In [0]:
# Upload documents to Alab

alab.upload_preannotations(
  project_name = 'suicide_detection',
  preannotations = pre_annotations[:5]) # testing with 5 annotations

Uploading 5 preannotation(s).
Operation completed successfully. Response code: 201
Out[23]: {'completion_count': 0,
 'duration': 0.28943872451782227,
 'failed_count': 0,
 'ignored_count': 0,
 'prediction_count': 5,
 'task_count': 5,
 'task_ids': [1, 2, 3, 4, 5],
 'task_title_warning': 0,
 'updated_count': 0}

# Annotate Documents on Annotation Lab and import

Import Annotations

In [0]:
alab.get_annotations(
  project_name = 'suicide_detection', 
  output_name='result.json',
  save_dir=delta_silver_path)


Operation completed successfully. Response code: 200

In [0]:
dbutils.fs.ls(delta_silver_path)

Out[24]: [FileInfo(path='dbfs:/FileStore/HLS/nlp/delta/silver/result.json', name='result.json', size=16912)]

## Convert Export to Conll file

In [0]:
alab.get_conll_data(spark, f"/dbfs/{delta_silver_path}result.json", output_name='conll_demo', save_dir=f"/dbfs/{delta_silver_path}")

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][OK!]
Spark NLP LightPipeline is created
sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[ | ][OK!]
Spark NLP LightPipeline is created
Attempting to process: Task ID# 1
Task ID# 1 is included
Attempting to process: Task ID# 2
Task ID# 2 is included
Attempting to process: Task ID# 3
Task ID# 3 is included
Attempting to process: Task ID# 4
Task ID# 4 is included
Attempting to process: Task ID# 5
Task ID# 5 is included
Saved in location: /dbfs//FileStore/HLS/nlp/delta/silver//conll_demo.conll

Printing first 30 lines of CoNLL for inspection:

Out[27]: ['-DOCSTART- -X- -1- O\n\n',
 'I MC MC O\n',
 'really RR RR O\n',
 "don't JJ JJ B-SUICIDE_BEHAVIOR\n",
 'want JJ JJ I-SUICIDE_BEHAVIOR\n',
 'to II II I-SUICIDE_BEHAVIOR\n',
 'live JJ JJ I-SUICIDE_BEHAVIOR\n',
 'anymore NN NN O\n',
 '- SYM SYM O\n',
 "i've NN NN O\n",
 'never RR RR O\n',
 'talked VVN VVN O\n',
 'about II II O\n',
 'this DD DD O\n',
 'with II II O\n',
 'anyone NN NN O\n',
 ', NN NN O\n',
 'but CC CC O\n',
 'i MC MC O\n',
 'really RR RR O\n',
 "don't JJ JJ B-SUICIDE_BEHAVIOR\n",
 'want JJ JJ I-SUICIDE_BEHAVIOR\n',
 'to II II I-SUICIDE_BEHAVIOR\n',
 'live JJ JJ I-SUICIDE_BEHAVIOR\n',
 'anymore NN NN O\n',
 '. NN NN O\n',
 '\n',
 'because CS CS O\n',
 'what PND PND O\n',
 'is VBZ VBZ O\n']

In [0]:
dbutils.fs.ls(delta_silver_path)

Out[28]: [FileInfo(path='dbfs:/FileStore/HLS/nlp/delta/silver/conll_demo.conll', name='conll_demo.conll', size=3760),
 FileInfo(path='dbfs:/FileStore/HLS/nlp/delta/silver/result.json', name='result.json', size=16912)]

## Train NER Model

Load Data

In [0]:
conll_data = CoNLL().readDataset(spark, f"{delta_silver_path}conll_demo.conll")

conll_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| pos| label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
I really don't wa...|[{document, 0, 12...|[{document, 0, 12...|[{token, 0, 0, I,...|[{pos, 0, 0, MC, ...|[{named_entity, 0...|
because what is t...|[{document, 0, 26...|[{document, 0, 26...|[{token, 0, 6, be...|[{pos, 0, 6, CS, ...|[{named_entity, 0...|
why am i here ? i...|[{document, 0, 16...|[{document, 0, 16...|[{token, 0, 2, wh...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows

In [0]:
clinical_embeddings = WordEmbeddingsModel.pretrained('embeddings_clinical', "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
graph_folder_path = "/dbfs/ner/medical_ner_graphs"

ner_graph_builder = TFGraphBuilder()\
    .setModelName("ner_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("label")\
    .setGraphFolder(graph_folder_path)\
    .setGraphFile("auto")\
    .setHiddenUnitsNumber(20)\
    .setIsMedical(True) # False -> if you want to use TFGraphBuilder with NerDLApproach

In [0]:
nerTagger = MedicalNerApproach()\
  .setInputCols(["sentence", "token", "embeddings"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setMaxEpochs(20)\
  .setBatchSize(64)\
  .setRandomSeed(0)\
  .setVerbose(1)\
  .setValidationSplit(0.5)\
  .setEvaluationLogExtended(True) \
  .setEnableOutputLogs(True)\
  .setIncludeConfidence(True)\
  .setOutputLogsPath('dbfs:/ner/ner_logs')\
  .setUseBestModel(True)\
  .setGraphFolder('dbfs:/ner/medical_ner_graphs')\
  .setValidationSplit(0.2)
  # .setEnableMemoryOptimizer(True) #>> if you have a limited memory and a large conll file, you can set this True to train batch by batch       

ner_pipeline = Pipeline(stages=[
          clinical_embeddings,
          ner_graph_builder,
          nerTagger
 ])

In [0]:
model = ner_pipeline.fit(conll_data)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: /dbfs/ner/medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 37, 'is_medical': True, 'lstm_size': 20}
ner_dl graph exported to /dbfs/ner/medical_ner_graphs/blstm_3_200_20_37.pb

In [0]:
with open('/dbfs/ner/ner_logs/MedicalNerApproach_376500b42b33.log', 'r') as f_:
  lines = ''.join(f_)
print (lines)

Name of the selected graph: /databricks/driver/file:/local_disk0/tmp/sparknlp_tmp_6077641227228493032/blstm_3_200_20_37.pb
Training started - total epochs: 20 - lr: 0.001 - batch size: 64 - labels: 3 - chars: 36 - training examples: 15


Epoch 1/20 started, lr: 0.001, dataset size: 15


Epoch 1/20 - 1.46s - loss: 64.84794 - avg training loss: 16.211985 - batches: 4
Quality on validation dataset (20.0%), validation examples = 3
time to finish evaluation: 0.38s
Total validation loss: 4.4897	Avg validation loss: 4.4897
label tp fp fn prec rec f1
I-SUICIDE_BEHAVIOR 0 1 0 0.0 0.0 0.0
tp: 0 fp: 1 fn: 0 labels: 1
Macro-average prec: 0.0, rec: 0.0, f1: NaN
Micro-average prec: 0.0, rec: 0.0, f1: 0.0


Epoch 2/20 started, lr: 9.950249E-4, dataset size: 15


Epoch 2/20 - 0.05s - loss: 22.561287 - avg training loss: 5.6403217 - batches: 4
Quality on validation dataset (20.0%), validation examples = 3
time to finish evaluation: 0.00s
Total validation loss: 1.8988	Avg validation loss: 1.8988
label tp fp fn prec rec f1
tp: 0 fp: 0 fn: 0 labels: 0
Macro-average prec: NaN, rec: NaN, f1: NaN
Micro-average prec: 0.0, rec: 0.0, f1: 0.0


Epoch 3/20 started, lr: 9.90099E-4, dataset size: 15


Epoch 3/20 - 0.06s - loss: 18.072136 - avg training loss: 4.518034 - batches: 4
Quality on validation dataset (20.0%), validation examples = 3
time to finish evaluation: 0.00s
Total validation loss: 0.9263	Avg validation loss: 0.9263
label tp fp fn prec rec f1
tp: 0 fp: 0 fn: 0 labels: 0
Macro-average prec: NaN, rec: NaN, f1: NaN
Micro-average prec: 0.0, rec: 0.0, f1: 0.0


Epoch 4/20 started, lr: 9.852217E-4, dataset size: 15


Epoch 4/20 - 0.06s - loss: 18.778093 - avg training loss: 4.6945233 - batches: 4
Quality on validation dataset (20.0%), validation examples = 3
time to finish evaluation: 0.00s
Total validation loss: 0.5627	Avg validation loss: 0.5627
label tp fp fn prec rec f1
tp: 0 fp: 0 fn: 0 labels: 0
Macro-average prec: NaN, rec: NaN, f1: NaN
Micro-average prec: 0.0, rec: 0.0, f1: 0.0


Epoch 5/20 started, lr: 9.803922E-4, dataset size: 15


Epoch 5/20 - 0.06s - loss: 20.441402 - avg training loss: 5.1103506 - batches: 4
Quality on validation dataset (20.0%), validation examples = 3
time to finish evaluation: 0.00s
Total validation loss: 0.4262	Avg validation loss: 0.4262
label tp fp fn prec rec f1
tp: 0 fp: 0 fn: 0 labels: 0
Macro-average prec: NaN, rec: NaN, f1: NaN
Micro-average prec: 0.0, rec: 0.0, f1: 0.0


Epoch 6/20 started, lr: 9.756098E-4, dataset size: 15


Epoch 6/20 - 0.05s - loss: 19.015844 - avg training loss: 4.753961 - batches: 4
Quality on validation dataset (20.0%), validation examples = 3
time to finish evaluation: 0.00s
Total validation loss: 0.3867	Avg validation loss: 0.3867
label tp fp fn prec rec f1
tp: 0 fp: 0 fn: 0 labels: 0
Macro-average prec: NaN, rec: NaN, f1: NaN
Micro-average prec: 0.0, rec: 0.0, f1: 0.0


Epoch 7/20 started, lr: 9.7087387E-4, dataset size: 15


Epoch 7/20 - 0.06s - loss: 17.689075 - avg training loss: 4.422269 - batches: 4
Quality on validation dataset (20.0%), validation examples = 3
time to finish evaluation: 0.00s
Total validation loss: 0.3937	Avg validation loss: 0.3937
label tp fp fn prec rec f1
tp: 0 fp: 0 fn: 0 labels: 0
Macro-average prec: NaN, rec: NaN, f1: NaN
Micro-average prec: 0.0, rec: 0.0, f1: 0.0


Epoch 8/20 started, lr: 9.6618367E-4, dataset size: 15


Epoch 8/20 - 0.06s - loss: 15.639478 - avg training loss: 3.9098694 - batches: 4
Quality on validation dataset (20.0%), validation examples = 3
time to finish evaluation: 0.00s
Total validation loss: 0.4315	Avg validation loss: 0.4315
label tp fp fn prec rec f1
tp: 0 fp: 0 fn: 0 labels: 0
Macro-average prec: NaN, rec: NaN, f1: NaN
Micro-average prec: 0.0, rec: 0.0, f1: 0.0


Epoch 9/20 started, lr: 9.615385E-4, dataset size: 15


Epoch 9/20 - 0.06s - loss: 14.217161 - avg training loss: 3.5542903 - batches: 4
Quality on validation dataset (20.0%), validation examples = 3
time to finish evaluation: 0.00s
Total validation loss: 0.4849